In [ ]:
!git clone https://github.com/MaybeShewill-CV/lanenet-lane-detection.git


In [2]:
!cd E:\Codes\smart_traffic\lanenet-lane-detection


In [19]:
import tensorflow as tf
import cv2
import numpy as np

# Disable eager execution to use TensorFlow 1.x syntax
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto(device_count={'GPU': 0})

In [20]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4351734664601887186
xla_global_id: -1
]


In [14]:
def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (512, 256))

    # Convert the image from BGR to RGB (OpenCV loads images in BGR format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Normalize pixel values to the range expected by the model (e.g., 0 to 1)
    image = image / 255.0

    # Expand dimensions to match the expected batch size (1, height, width, channels)
    image = np.expand_dims(image, axis=0)

    return image

In [15]:
import cv2
import numpy as np

def process_output(binary_segmentation, instance_segmentation, original_image):

    # Resize binary_segmentation to match original image size
    binary_segmentation_resized = cv2.resize(binary_segmentation, 
                                             (original_image.shape[1], original_image.shape[0]),
                                             interpolation=cv2.INTER_NEAREST)

    # Threshold binary segmentation to create a binary mask
    binary_mask = (binary_segmentation_resized > 0.5).astype(np.uint8) * 255

    # Overlay binary mask on the original image
    overlay = original_image.copy()
    overlay[binary_mask == 255] = [0, 255, 0]  # Green color for lanes

    # Instance segmentation (optional): further refine lane detection per instance
    instance_segmentation_resized = cv2.resize(instance_segmentation, 
                                               (original_image.shape[1], original_image.shape[0]),
                                               interpolation=cv2.INTER_NEAREST)
    
    # Apply clustering or contouring (depends on model output) for unique lane representation
    # Here we extract contours for each unique lane if possible
    num_lanes = np.unique(instance_segmentation_resized)
    for lane_id in num_lanes:
        if lane_id == 0:  # skip background
            continue
        lane_mask = (instance_segmentation_resized == lane_id).astype(np.uint8)
        
        # Find contours to draw lane line
        contours, _ = cv2.findContours(lane_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, contours, -1, (0, 0, 255), 2)  # Red color for unique lanes

    # Combine overlay with original image for visualization
    output_image = cv2.addWeighted(original_image, 0.7, overlay, 0.3, 0)

    return output_image


In [17]:
image_path = "C:\\Users\\tyagi\\OneDrive\\Desktop\\Images Output\\drone\\seq3-drone_0000001.jpg"
with tf.device('/cpu:0'):
    with tf.compat.v1.Session(config=config) as sess:
        # Define the checkpoint path
        ckpt_path = "E:\\Codes\\smart_traffic\\lanenet-lane-detection\\checkpoint\\tusimple_lanenet.ckpt"

        # Load the model graph
        saver = tf.compat.v1.train.import_meta_graph(ckpt_path + '.meta', clear_devices=True)

        # Restore the weights from the checkpoint
        saver.restore(sess, ckpt_path)

        # Get the input and output tensors by their names from the graph
        graph = tf.compat.v1.get_default_graph()
        input_tensor = graph.get_tensor_by_name('input_tensor_name:0')  # Replace with the actual input tensor name
        output_tensor = graph.get_tensor_by_name('output_tensor_name:0')  # Replace with the actual output tensor name

        # Preprocess the input image as required by the model
        input_image = preprocess_image(image_path)  # Implement this function as needed

        # Run inference
        output = sess.run(output_tensor, feed_dict={input_tensor: input_image})

        # Process and visualize the output
        process_output(output)  


'metric_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from E:\Codes\smart_traffic\lanenet-lane-detection\checkpoint\tusimple_lanenet.ckpt


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Graph execution error:

Detected at node 'LaneNet/bisenetv2_frontend/detail_branch/stage_1/conv_block_1_repeat_1/3x3_conv/conv/W/Initializer/truncated_normal/TruncatedNormal' defined at (most recent call last):
Node: 'LaneNet/bisenetv2_frontend/detail_branch/stage_1/conv_block_1_repeat_1/3x3_conv/conv/W/Initializer/truncated_normal/TruncatedNormal'
Cannot assign a device for operation LaneNet/bisenetv2_frontend/detail_branch/stage_1/conv_block_1_repeat_1/3x3_conv/conv/W/Initializer/truncated_normal/TruncatedNormal: {{node LaneNet/bisenetv2_frontend/detail_branch/stage_1/conv_block_1_repeat_1/3x3_conv/conv/W/Initializer/truncated_normal/TruncatedNormal}} was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhos